# Turnaj: kámen-nůžky-papír

## Zadání
Vyzkoušejte kdo by vyhrál z následujících hráčů turnaj ve hře kámen-nůžky-papír. Hráč je definován jménem a pak strategií (pravděpodobnosti co zahraje).

Otestujte na velkem množství her.

In [1]:
# the task definition
PLAYERS = {
    #name : (rock, scissors, paper)
    "GTO": (1/3., 1/3., 1/3,),
    "Geologist": (0.8, 0.1, 0.1),
    "Tailor": (0.1, 0.8, 0.1),
    "Academic": (0.1, 0.1, 0.8),
    "Regular 1": (0.3, 0.5, 0.2),
    "Regular 2": (0.2, 0.6, 0.2),
    "Regular 3": (0.5, 0.4, 0.1),
}

## Pravidla

In [2]:
def apply_rules(hero, villain):
    WINS = {
        "rock": "scissors",
        "scissors": "paper",
        "paper": "rock",
    }
    if WINS[hero] == villain:
        return 1
    elif WINS[villain] == hero:
        return -1
    else:
        return 0
        
TESTS = (
    ("rock", "rock"),
    ("scissors", "rock"),
    ("paper", "rock"),
)
            
for test in TESTS:
    print(apply_rules(*test))

0
-1
1


## Model hráče

In [3]:
import random

class Player():
    
    def __init__(self, name, strategy):
        self.name = name
        self.choice = "rock"
        self.strategy = strategy
        self.wins = 0
        self.losses = 0
        self.win_rate = False

    def update_score(self, value):
        if value > 0:
            self.wins += 1
        elif value < 0:
            self.losses += 1
        if self.losses:
            self.win_rate = self.wins / self.losses

    def update(self, value, opponent):
        self.update_score(value)
        
    def make_choice(self):
        r = random.random()
        if self.strategy[0] > r:
            self.choice = "rock"
        elif self.strategy[0] + self.strategy[1] > r:
            self.choice = "scissors"
        else:
            self.choice = "paper"            
        
    def __str__(self):
        return "{}{}\t{}\t{}\t{}".format(self.name.ljust(15), round(self.win_rate, 3), self.wins, self.losses, self.strategy)
    
    def __repr__(self):
        return self.__str__()

## Turnaj

In [4]:
import itertools

def full_round_robin(players):
    matches = itertools.combinations(players, r=2)
    for match in matches:
        for p in match:
            p.make_choice()        
        p1, p2 = match
        outcome = apply_rules(p1.choice, p2.choice)
        p1.update(outcome, p2.name)
        p2.update(-outcome, p1.name)    
        
players = []
for key, value in PLAYERS.items():
    p = Player(key, value)
    players.append(p)
    
for _ in range(10000):    
    full_round_robin(players)

players.sort(key=lambda p: p.win_rate, reverse=True)
for p in players:
    print(p)


Geologist      1.376	25035	18190	(0.8, 0.1, 0.1)
Regular 3      1.138	21028	18476	(0.5, 0.4, 0.1)
GTO            1.008	20195	20036	(0.3333333333333333, 0.3333333333333333, 0.3333333333333333)
Regular 1      0.975	19251	19738	(0.3, 0.5, 0.2)
Regular 2      0.915	18719	20467	(0.2, 0.6, 0.2)
Academic       0.85	21495	25298	(0.1, 0.1, 0.8)
Tailor         0.838	18166	21684	(0.1, 0.8, 0.1)


## Průměrná strategie hráčů

In [5]:
default = [0, 0, 0]
for p in players:
    default = [default[_] + p.strategy[_] for _ in range(3)]
default = [_ / len(players) for _ in default]

print(default)

[0.33333333333333337, 0.4047619047619047, 0.2619047619047619]


## Vylepšený hráč

In [6]:
class Hero(Player):
    
    def __init__(self, *args):
        super().__init__(*args)
        self.history = {
            "rock": 0,
            "scissors": 0,
            "paper": 0,
        }
        
    def update(self, value, opponent):
        self.update_score(value)
        WINS = {
            "rock": "scissors",
            "scissors": "paper",
            "paper": "rock",
        }
        LOSS = {
            "rock": "paper",
            "scissors": "rock",
            "paper": "scissors",
        }
        if value > 0:
            key = WINS[self.choice]
        elif value < 0:
            key = LOSS[self.choice]
        else:
            key = self.choice
        self.history[key] += 1
        most_common = max(self.history, key=self.history.get)
        if most_common == "rock":
            self.strategy = (0, 0, 1)
        elif most_common == "scissors":
            self.strategy = (1, 0, 0)
        else:
            self.strategy = (0, 1, 0)
          
            
h = Hero("Hero", (1/3., 1/3., 1/3.))
for _ in range(10):
    h.update(0, "Test")
    
print(h.strategy)

(0, 0, 1)


In [7]:
players = []
for key, value in PLAYERS.items():
    p = Player(key, value)
    players.append(p)
   
h = Hero("Hero", (1/3., 1/3., 1/3.))
players.append(h)
    
for _ in range(10000):    
    full_round_robin(players)

players.sort(key=lambda p: p.win_rate, reverse=True)
for p in players:
    print(p)

Hero           1.538	28087	18260	(1, 0, 0)
Geologist      1.375	26118	18997	(0.8, 0.1, 0.1)
Academic       1.145	29736	25980	(0.1, 0.1, 0.8)
Regular 3      0.993	22020	22166	(0.5, 0.4, 0.1)
GTO            0.991	23034	23232	(0.3333333333333333, 0.3333333333333333, 0.3333333333333333)
Regular 1      0.846	20976	24797	(0.3, 0.5, 0.2)
Regular 2      0.765	20409	26669	(0.2, 0.6, 0.2)
Tailor         0.652	19233	29512	(0.1, 0.8, 0.1)


In [8]:
players = []
for key, value in PLAYERS.items():
    p = Player(key, value)
    players.append(p)

for idx in range(5):
    h = Hero("Hero" + str(idx), (1/3., 1/3., 1/3.))
    players.append(h)
    
for _ in range(10000):    
    full_round_robin(players)

players.sort(key=lambda p: p.win_rate, reverse=True)
for p in players:
    print(p)
    

Hero1          1.062	41309	38907	(0, 0, 1)
Hero4          1.061	40609	38281	(0, 0, 1)
Academic       1.054	39626	37613	(0.1, 0.1, 0.8)
Geologist      1.001	37799	37779	(0.8, 0.1, 0.1)
Hero2          0.998	38992	39079	(1, 0, 0)
Hero0          0.994	38760	38991	(1, 0, 0)
GTO            0.989	36436	36829	(0.3333333333333333, 0.3333333333333333, 0.3333333333333333)
Regular 3      0.981	36586	37301	(0.5, 0.4, 0.1)
Regular 1      0.978	36538	37347	(0.3, 0.5, 0.2)
Regular 2      0.97	36830	37985	(0.2, 0.6, 0.2)
Tailor         0.959	37806	39403	(0.1, 0.8, 0.1)
Hero3          0.956	38470	40246	(0, 0, 1)
